In [1]:
"""
Implementing Classifier with Neural Network and Regression for given dataset
"""
__author__ = ["Gahan Saraiya", "Priyanka Bhati"]

In [2]:
# Import built-in modules
import os
import numpy as np # linear algebra
import itertools
from subprocess import check_output
from collections import Counter

In [3]:
# Import 3rd party Python packages
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #for plotting
from sklearn import linear_model
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
# dividing into train and test
from sklearn.model_selection import train_test_split
import seaborn as sns
# print(check_output(["ls", "input"]).decode("utf8"))
# %matplotlib inline

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
batch_size = 64
num_classes = 10
epochs = 20
input_shape = (28, 28, 1)

Using TensorFlow backend.


In [5]:
WORKING_DIR = os.getcwd()
DATASET_DIR = os.path.join(WORKING_DIR, "dataset")
DATA_PATH_B = os.path.join(DATASET_DIR, "B.csv")
DATA_PATH_navy = os.path.join(DATASET_DIR, "navy_svm.csv")

In [6]:
# read data frame
# bdf = pd.read_csv(DATA_PATH_B)
navyData = pd.read_csv(DATA_PATH_navy)
navyData.head()

,class,a1,a2,a3,a4,a5,a6
0,0,235.47,12.22,0.271,25.2,45.6,25.28
1,0,235.47,12.22,0.271,25.2,45.6,25.28
2,0,235.66,12.22,1.061,25.2,45.6,25.28
3,0,235.66,12.11,1.061,25.2,45.6,25.28
4,0,235.66,12.06,1.341,25.2,45.6,25.28


In [7]:
navyData.describe()

,class,a1,a2,a3,a4,a5,a6
count,582905.000000,582905.000000,582905.000000,582905.000000,582905.000000,582905.000000,582905.000000
mean,0.283895,232.478022,11.897178,0.784452,21.182873,38.854022,27.888130
std,0.450887,4.017430,0.372777,0.410430,3.781182,4.108154,1.400513
min,0.000000,217.590000,0.160000,0.031000,16.400000,27.900000,-11.550000
25%,0.000000,229.670000,11.640000,0.451000,18.400000,36.800000,26.880000
50%,0.000000,233.740000,11.860000,0.681000,20.100000,39.100000,27.540000
75%,1.000000,235.700000,12.080000,1.051000,22.700000,40.600000,28.670000
max,1.000000,244.600000,16.330000,5.781000,39.300000,52.700000,37.130000


In [8]:
navyData.groupby('class').mean()

,a1,a2,a3,a4,a5,a6
class,,,,,,
0,232.752346,11.800671,0.747009,19.535520,39.021003,27.171677
1,231.786060,12.140608,0.878898,25.338199,38.432825,29.695329


In [9]:
##Saperating attributes and labels
navy_data_attrib = navyData.iloc[:, 1:]
navy_data_class = navyData.iloc[:, [0]] # preserves column header rather than converting to series navyData['class']
attrib_train, attrib_test, class_train, class_test = train_test_split(
    navy_data_attrib, 
    navy_data_class,
#     train_size=0.085778,  # Exact 100000 for train size 0.171556
#     test_size=0.4,
    shuffle=True, # boolean, optional
    stratify=None # array-like or None: If not None, data is split in a stratified fashion, using this as the class labels.
)

## Define Model in Keras  

1. Models in Keras are defined as a sequence of layers
2. We create a Sequential model and add layers one at a time until we are happy with our network topology.
3. The first thing to get right is to ensure the input layer has the right number of inputs. This can be specified when creating the first layer with the input_dim argument and setting it to 6 for the 6 input variables.

### Deciding the number of layers and their types
There are heuristics that we can use and often the best network structure is found through a process of trial and error experimentation. Generally, you need a network large enough to capture the structure of the problem if that helps at all.

> Here We'll first use fully connected net.

Fully connected layers are defined using the `Dense` class. We can specify the number of neurons in the layer as the first argument, the initialization method as the second argument as init and specify the activation function using the activation argument.

Below displayed a list of comparions for activation functions:

![activation_function_comparison](activation_function_comparison.png)

In [10]:
# create model
model = Sequential()
first_layer_neurons = 12
input_dimension = 6
model.add(Dense(first_layer_neurons, input_dim=input_dimension, activation='relu'))
second_layer_neurons = 8
model.add(Dense(second_layer_neurons, activation='relu'))
num_classes = 1
model.add(Dense(num_classes, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


## Compiling the model  

Compiling the model uses the efficient numerical libraries under the covers (the so-called backend) such as Theano or TensorFlow. The backend automatically chooses the best way to represent the network for training and making predictions to run on your hardware, such as CPU or GPU or even distributed.


In this case, we will use logarithmic loss, which for a binary classification problem is defined in Keras as `binary_crossentropy`. We will also use the efficient gradient descent algorithm `adam` for no other reason that it is an efficient default. Learn more about the Adam optimization algorithm in the paper [Adam: A Method for Stochastic Optimization](http://arxiv.org/abs/1412.6980).

In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Fit Model  
time to execute the model on some data!  

We can train or fit our model on our loaded data by calling the `fit()` function on the model.

The training process will run for a fixed number of iterations through the dataset called epochs, that we must specify using the `nepochs` argument. We can also set the number of instances that are evaluated before a weight update in the network is performed, called the batch size and set using the `batch_size` argument.

## Evaluate Model  

This will only give us an idea of how well we have modeled the dataset (e.g. train accuracy), but no idea of how well the algorithm might perform on new data. We have done this for simplicity, but ideally, you could separate your data into train and test datasets for training and evaluation of your model.

`evaluate()` - function on model to be used to evaluate.
          pass it the same input and output used to train the model.

In [13]:
for iteration in [20, 50]:
    # Fit the model
    model.fit(attrib_train, class_train, epochs=iteration, batch_size=100)
    # evaluate the model
    scores = model.evaluate(attrib_train, class_train)
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/20
437178/437178 [==============================] - 4s 10us/step - loss: 4.5756 - acc: 0.7161
Epoch 2/20
257000/437178 [================>.............] - ETA: 1s - loss: 4.5871 - acc: 0.7154

KeyboardInterrupt: 

In [15]:
# create model
model = Sequential()
first_layer_neurons = 128
input_dimension = 6
model.add(Dense(first_layer_neurons, input_dim=input_dimension, activation='relu'))
# model.add(Dropout(0.20))
second_layer_neurons = 64
model.add(Dense(second_layer_neurons, activation='relu'))
num_classes = 1
model.add(Dense(num_classes, activation='relu'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
learning_rate_reduction = ReduceLROnPlateau(monitor='acc', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

for iteration in [20]:
    # Fit the model
    model.fit(attrib_train, 
              class_train, 
              epochs=iteration, 
              batch_size=100,
              callbacks=[learning_rate_reduction]
             )
    # evaluate the model
    scores = model.evaluate(attrib_train, class_train)
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/20
437178/437178 [==============================] - 6s 14us/step - loss: 4.5756 - acc: 0.7161
Epoch 2/20
437178/437178 [==============================] - 6s 14us/step - loss: 4.5756 - acc: 0.7161
Epoch 3/20
437178/437178 [==============================] - 6s 13us/step - loss: 4.5756 - acc: 0.7161
Epoch 4/20
437178/437178 [==============================] - 5s 12us/step - loss: 4.5756 - acc: 0.7161
Epoch 5/20
437178/437178 [==============================] - 5s 12us/step - loss: 4.5756 - acc: 0.7161
Epoch 6/20
437178/437178 [==============================] - 6s 13us/step - loss: 4.5756 - acc: 0.7161

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 7/20
437178/437178 [==============================] - 5s 12us/step - loss: 4.5756 - acc: 0.7161
Epoch 8/20
437178/437178 [==============================] - 6s 13us/step - loss: 4.5756 - acc: 0.7161
Epoch 9/20
437178/437178 [==============================] - 6s 13us/step - loss: 4.5756 - acc: 0.7161
E

In [ ]:
predictions = model.predict(attrib_train)